In [ ]:
from __future__ import annotations
import typing
import json
import pathlib
import os
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_

import transformers
import transformers.modeling_outputs
from transformers import AutoTokenizer, AutoModel

from nltk.corpus import stopwords

import sklearn
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

from tqdm import tqdm

In [ ]:
IS_KAGGLE = "KAGGLE_DOCKER_IMAGE" in os.environ

DATASETS = pathlib.Path(
    "."
    if not IS_KAGGLE
    else "/kaggle/input/influencers-or-observers-predicting-social-roles/Kaggle2025"
)

DATASET_TRAIN = DATASETS / "train.jsonl"
DATASET_KAGGLE = DATASETS / "kaggle_test.jsonl"

CACHE_DIR = pathlib.Path(".")

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# TODO: Maybe make deterministic? A bit too late though, we already have trained a checkpoint with randomness
# torch.random.manual_seed(42)
# np.random.seed(42)

# Data loading

In [4]:
def load_json(path: pathlib.Path, cache: bool = False) -> pd.DataFrame:
    path_pq = (CACHE_DIR / path.name).with_stem(f"{path.stem}_raw").with_suffix(".parquet")
    
    if cache and path_pq.exists():
        return pd.read_parquet(path_pq)
    
    # This leaves things to be desired, since there's no way to specify dtypes
    # and it assumes float instead of int, causing a loss in precision...
    # But I guess it only matters for ids, which we'll probably discard in preprocessing anyway
    result = pd.json_normalize(list(map(json.loads, path.read_bytes().splitlines())))
    
    if cache:
        result.to_parquet(path_pq)
    
    return result


In [5]:
train_data = load_json(DATASET_TRAIN, cache=True)
kaggle_data = load_json(DATASET_KAGGLE, cache=True)

# Preprocessing

In [6]:

def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df["is_reply"] = df["in_reply_to_status_id"].notna()
    
    # Remove various ID fields
    # TODO: Could we actually use them for something?
    # Note: challenge_id and label seem to be added for the kaggle challenge
    df = df.drop(columns=[
        "in_reply_to_status_id_str",
        "in_reply_to_status_id",
        "in_reply_to_user_id_str",
        "in_reply_to_user_id",
        "quoted_status_id_str",
        "quoted_status_id",
        "id_str",
        "quoted_status.in_reply_to_status_id_str",
        "quoted_status.in_reply_to_status_id",
        "quoted_status.in_reply_to_user_id_str",
        "quoted_status.in_reply_to_user_id",
        "quoted_status.id_str",
        "quoted_status.id",
        "quoted_status.user.id_str",
        "quoted_status.user.id",
        "quoted_status_permalink.expanded",
        "quoted_status_permalink.display",
        "quoted_status_permalink.url",
        "quoted_status.quoted_status_id",
        "quoted_status.quoted_status_id_str",
        "quoted_status.place.id",
        "place.id",
        "lang",  # Always "fr"
        "retweeted",  # Always False
        "filter_level",  # Always "low"
        "geo",  # Always None
        "place",  # Always None
        "coordinates",  # Always None
        "contributors",  # Always None
        "quote_count",  # Always 0
        "reply_count",  # Always 0
        "retweet_count",  # Always 0
        "favorite_count",  # Always 0
        "favorited",  # Always False
        "quoted_status.geo",  # Always None
        "quoted_status.place",  # Always None
        "quoted_status.coordinates",  # Always None
        "quoted_status.retweeted",  # Always False
        "quoted_status.filter_level",  # Always "low"
        "quoted_status.contributors",  # Always None
        "quoted_status.user.utc_offset",  # Always None
        "quoted_status.user.lang",  # Always None
        "quoted_status.user.time_zone",  # Always None
        "quoted_status.user.follow_request_sent",  # Always None
        "quoted_status.user.following",  # Always None
        "quoted_status.user.notifications",  # Always None
        "user.default_profile_image",  # Always False
        "user.protected",  # Always False
        "user.contributors_enabled",  # Always False
        "user.lang",  # Always None
        "user.notifications",  # Always None
        "user.following",  # Always None
        "user.utc_offset",  # Always None
        "user.time_zone",  # Always None
        "user.follow_request_sent",  # Always None
    ])
    
    # TODO: Augment text with other string features?
    df["full_text"] = df.apply(lambda tweet: extract_full_text(tweet), axis=1)
    
    return df


def extract_full_text(tweet: pd.Series) -> str:
    text: str = tweet["text"]
    
    if not pd.isna(tweet["extended_tweet.full_text"]):
        text = tweet["extended_tweet.full_text"]
    
    return text


In [7]:
X_train = train_data.drop("label", axis=1)
y_train = train_data["label"]

X_kaggle = kaggle_data

X_train = preprocess(X_train)
X_kaggle = preprocess(X_kaggle)

# Models

In [8]:
# TODO: discard quoted_status.lang != "fr"?
# TODO: some tweets are images

In [ ]:

NUM_CLASSES = 2

class TweetClassifier(nn.Module):
    tokenizer: nn.Module
    encoder: nn.Module
    metadata_dim: int
    md_layernorm: nn.Module
    fc1: nn.Module
    fc2: nn.Module
    
    def __init__(
        self,
        pretrained_encoder: str = "distilbert-base-cased", # "camembert-base", "Geotrend/distilbert-base-en-fr-cased", "flaubert/flaubert_base_cased", "flaubert/flaubert_small_cased"
        metadata_dim: int = 16,
        hidden_dim: int = 128,
        max_length: int = 256,
    ):
        super().__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_encoder)
        self.encoder = AutoModel.from_pretrained(pretrained_encoder)

        # Don't finetune the encoder... yet?
        for param in self.encoder.parameters():
            param.requires_grad = False
        
        self.encoder_dim = self.encoder.config.hidden_size
        self.max_length = max_length

        self.metadata_dim = metadata_dim
        self.md_layernorm = nn.LayerNorm(metadata_dim)
        
        self.fc1 = nn.Linear(self.encoder_dim + metadata_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, NUM_CLASSES)
    
    @property
    def device(self) -> torch.device:
        return next(self.parameters()).device
    
    def encode_text(self, texts: list[str]) -> torch.Tensor:
        encoded: torch.Tensor = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        ).to(self.device)

        outputs: transformers.modeling_outputs.BaseModelOutput = self.encoder(**encoded)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
        return cls_embeddings  # [batch, encoder_dim]
    
    def forward(
        self,
        texts: list[str] | torch.Tensor,
        metadata: torch.Tensor,
    ) -> dict[str, torch.Tensor]:
        """
        Returns dict with:
            "logits": tensor [batch_size, num_classes]
            "probs": tensor [batch_size, num_classes]
        """
        device = self.device
        batch_size = len(texts)
        
        if isinstance(texts, torch.Tensor):
            text_vecs = texts
        else:
            text_vecs = self.encode_text(texts)  # [B, encoder_dim]

        metadata = metadata.to(device)
        assert metadata.shape == (batch_size, self.metadata_dim)
        
        metadata = self.md_layernorm(metadata)
        
        x = torch.cat([text_vecs, metadata], dim=1)

        hidden = F.relu(self.fc1(x))
        logits = self.fc2(hidden)
        probs = F.softmax(logits, dim=-1)

        return {
            "logits": logits,
            "probs": probs,
        }


In [17]:
METADATA_DIM = 13  # Note: update when adding new fields!!

def extract_metadata(df: pd.DataFrame) -> torch.Tensor:
    md: list[pd.Series] = []

    def md_add_bool(col: str):
        md.append(df[col].apply(lambda x: (1 if x else -1) if pd.notnull(x) else 0))

    md_add_bool("is_quote_status")
    md_add_bool("is_reply")
    md_add_bool("possibly_sensitive")
    md_add_bool("quoted_status.user.verified")

    def md_add_len(col: str):
        # pd.notnull considered lists as not scalar
        md.append(df[col].apply(lambda x: len(x) if x is not None and not (isinstance(x, float) and np.isnan(x)) else 0))

    md_add_len("full_text")
    md_add_len("extended_tweet.entities.urls")
    md_add_len("extended_tweet.entities.hashtags")
    md_add_len("extended_tweet.entities.user_mentions")
    md_add_len("extended_tweet.entities.symbols")

    def md_add_time(col: str):
        tmp = df[col].apply(lambda x: time.mktime(time.strptime(x, "%a %b %d %H:%M:%S %z %Y")) if pd.notnull(x) else pd.NA)
        md.append(tmp.fillna(tmp.mean()))

    md_add_time("created_at")  # TODO: Same as timestamp_ms / 1000?
    md_add_time("quoted_status.user.created_at")

    def md_add_num(col: str):
        tmp = df[col].apply(pd.to_numeric)
        md.append(tmp.fillna(tmp.mean()))

    md_add_num("quoted_status.user.followers_count")
    md_add_num("timestamp_ms")

    return torch.from_numpy(np.array(md)).transpose(0, 1).float()


In [18]:
class TweetDataset(Dataset):
    texts: list[str]
    metadata: torch.Tensor
    labels: torch.Tensor
    device: torch.device
    
    def __init__(self, df: pd.DataFrame, labels: pd.Series, device: torch.device = device):
        self.texts = df["full_text"].tolist()
        self.metadata = extract_metadata(df).to(device)
        self.labels = torch.tensor(labels, dtype=torch.long, device=device)
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "text": self.texts[idx],
            "metadata": self.metadata[idx],
            "label": self.labels[idx],
        }


def collate_fn(batch):
    texts = [x["text"] for x in batch]
    metadata = torch.stack([x["metadata"] for x in batch])
    labels = torch.stack([x["label"] for x in batch])
    return texts, metadata, labels


In [ ]:
def train_model(
    model: TweetClassifier,
    train_ds: Dataset,
    epochs: int = 3,
    lr: float = 2e-4,
    weight_decay: float = 0.01,
    max_grad_norm: float = 1.0,
    device: torch.device = device,
    batch_size: int = 32,
) -> TweetClassifier:
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
    )
    
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}/{epochs}")
        model.train()
        total_loss = 0.0

        status_bar = tqdm(train_loader, desc="Training")

        for texts, metadata, labels in status_bar:
            texts: list[str]
            metadata: torch.Tensor
            labels: torch.Tensor
            
            metadata = metadata.to(device)
            labels = labels.to(device)

            optimizer.zero_grad(set_to_none=True)

            out = model(
                texts=texts,
                metadata=metadata,
            )
            logits = out["logits"]
            
            loss: torch.Tensor = criterion(logits, labels)
            loss.backward()
            clip_grad_norm_(model.parameters(), max_grad_norm)  # TODO: ?
            optimizer.step()

            total_loss += loss.item()
            status_bar.set_postfix({"loss": total_loss / (status_bar.n + 1)})

        print(f"Train Loss: {total_loss / len(train_loader):.4f}")

    return model


In [ ]:
def evaluate_model(
    model: TweetClassifier,
    val_ds: Dataset,
    device: torch.device = device,
    batch_size: int = 32,
) -> tuple[float, float]:
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
    )
    
    model.eval()
    criterion = torch.nn.CrossEntropyLoss()

    total_loss = 0.0
    correct = 0
    count = 0

    with torch.no_grad():
        status_bar = tqdm(val_loader, desc="Evaluating")
        
        for texts, metadata, labels in status_bar:
            texts: list[str]
            metadata: torch.Tensor
            labels: torch.Tensor
            metadata = metadata.to(device)
            labels = labels.to(device)

            out = model(
                texts=texts,
                metadata=metadata,
            )
            logits: torch.Tensor = out["logits"]
            
            loss: torch.Tensor = criterion(logits, labels)
            total_loss += loss.item()
            preds = logits.argmax(dim=-1)
            correct += (preds == labels).sum().item()
            count += labels.size(0)
            
            status_bar.set_postfix({"loss": total_loss / (status_bar.n + 1), "acc": correct / count})

    return {
        "loss": total_loss / len(val_loader),
        "acc": correct / count,
    }


In [32]:
def infer_with_model(
    model: TweetClassifier,
    df: pd.DataFrame,
    out_file: pathlib.Path | str | None = None,
    device: torch.device = device,
    batch_size: int = 32,
) -> torch.Tensor:
    data_loader = DataLoader(
        TweetDataset(df, torch.zeros(len(df), dtype=torch.long, device=device), device=device),
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
    )
    
    model.eval()
    
    predictions = torch.zeros(len(df), dtype=torch.long)
    cur_idx = 0
    
    with torch.no_grad():
        for texts, metadata, _ in tqdm(data_loader, desc="Inferring"):
            texts: list[str]
            metadata: torch.Tensor
            metadata = metadata.to(device)

            out = model(
                texts=texts,
                metadata=metadata,
            )
            logits: torch.Tensor = out["logits"].cpu()
            
            predictions[cur_idx:cur_idx+len(texts)] = logits.argmax(dim=-1)
            cur_idx += len(texts)
    
    if out_file is not None:
        output = pd.concat([df["challenge_id"], pd.DataFrame(predictions)], axis=1, ignore_index=True)
        output.columns = ["ID", "Prediction"]
        output.to_csv(out_file, index=False)
    
    return predictions


In [22]:
model = TweetClassifier(
    pretrained_encoder="camembert-base",
    metadata_dim=METADATA_DIM,
    hidden_dim=128,
    max_length=256
).to(device)

In [23]:
full_train_ds = TweetDataset(X_train, y_train, device=device)

train_ds, val_ds = random_split(full_train_ds, [0.9, 0.1])

C:\Users\Abel\AppData\Local\Temp\ipykernel_26644\54356709.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  md.append(tmp.fillna(tmp.mean()))


In [ ]:
def get_model_path(version: str) -> pathlib.Path:
    return pathlib.Path(
        f"./models/{version}/model-{version}.pt"
        if not IS_KAGGLE
        else f"/kaggle/input/model-{version}-pt/model-{version}.pt"
    )

In [ ]:
model_path = get_model_path("v1")

model.load_state_dict(torch.load(model_path, weights_only=True))

# model = train_model(model, train_ds, epochs=3, batch_size=64, device=device)
# torch.save(model.state_dict(), model_path)


In [ ]:
# TODO: Pre-encode all texts?
evaluate_model(model, val_ds, batch_size=64, device=device)

c:\Program Files\Python310\lib\site-packages\transformers\models\camembert\modeling_camembert.py:364: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


(0.595617502128955, 0.6738105997030533)

In [ ]:
infer_with_model(model, X_kaggle, batch_size=64, out_file="predictions-v1.csv", device=device)

C:\Users\Abel\AppData\Local\Temp\ipykernel_26644\54356709.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  md.append(tmp.fillna(tmp.mean()))
C:\Users\Abel\AppData\Local\Temp\ipykernel_26644\2988799321.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long, device=device)
Inferring: 100%|██████████| 1616/1616 [09:59<00:00,  2.70it/s]


tensor([1, 1, 0,  ..., 0, 0, 0])